<a href="https://colab.research.google.com/github/b4igiveup/ivy/blob/master/Original_data_facebook_All_layer_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fair-esm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import esm
import pandas as pd

layers=[0,15,33]
# Load ESM-1b model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

In [ ]:
import pathlib 
import numpy as np
workspace2="test"
np.set_printoptions(precision=3)
workspace="./"+workspace2
pathlib.Path(workspace+"Features").mkdir(parents=True, exist_ok=True) 

df_merge=pd.DataFrame()
with open(workspace2+"_fasta.txt") as fasta_file:
    
    flagpid=0
    for pid in fasta_file:

        pid=pid[1:-1]
        pid=pid.strip()
        fastseq=fasta_file.readline()
        print(pid)
        # print(fastseq[:-1])
        fasta=fastseq[:-1]

        print("Fasta length: ", len(fasta))

        if(len(fasta)>1022):
                n = 1022
                chunks = [fasta[i:i+n] for i in range(0, len(fasta), n)]
                print("No. of Chunks: ",len(chunks))

                flagc=0
                for chunk in chunks:
                    print("chunk length: ",len(chunk))
                    # print(chunk)

                    data = [ (pid, chunk)]
                    batch_labels, batch_strs, batch_tokens = batch_converter(data)
                    flagl=0
                    # print("Layer", layern)
                    layern=list(range(34)) 
                    with torch.no_grad():
                        results = model(batch_tokens, repr_layers=layern, return_contacts=True)
                    # Extract per-residue representations (on CPU)
                    for layern in range(34): #
                                                                     
                        token_representations = results["representations"][layern]
                        toekn=token_representations[0, 1 : len(chunk) + 1].numpy()
                        toekn=np.hstack((toekn,np.mean(toekn, axis=1).reshape(-1,1)))
                        if(flagl==0):
                            np_featurel=toekn
                            flagl=1
                        else:
                            np_featurel=np.hstack((np_featurel,toekn))                       
      

                if(flagc==0):
                    np_featurell=np_featurel
                    flagc=1
                else:
                    np_featurell=np.vstack((np_featurell,np_featurel))
                # print(np_featurell.shape)

        else:

            data = [ (pid, fasta)]
            batch_labels, batch_strs, batch_tokens = batch_converter(data)
            flagl=0
            layern=list(range(34))    
            # print("Layer", layern)
            # Extract per-residue representations (on CPU)
            with torch.no_grad():
                results = model(batch_tokens, repr_layers=layern, return_contacts=True)
     
                  
            for layern in range(34): #               
                
                token_representations = results["representations"][layern]
                toekn=token_representations[0, 1 : len(fasta) + 1].numpy()
                toekn=np.hstack((toekn,np.mean(toekn, axis=1).reshape(-1,1)))

                if(flagl==0):
                    np_featurell=toekn
                    flagl=1
                else:
                    np_featurell=np.hstack((np_featurell,toekn))                  
     
        
        if(flagpid==0):
            np_feature=np_featurell
            flagpid=1
        else:
            np_feature=np.vstack((np_feature,np_featurell))
        print(np_feature.shape) 

df_features=pd.DataFrame(np_feature)
df_features.columns= df_features.columns.astype(str)+"_FlModel"      
df_features.to_feather(workspace+"Features/"+pid+".ft")

DP01096
Fasta length:  2924
No. of Chunks:  3
chunk length:  1022
